# Evaluation of Results of "TUD-MMC at MediaEval 2016: Context of Experience task" by Wang & Liem

## Necessary imports

In [1]:
import pandas as pd
import numpy as np
import os.path
import xml.etree.ElementTree as ET

train_path = "res/coe_dataset_icpr/dev_set/"
test_path = "res/coe_dataset_icpr/test_set/"

audio_folder = "audio_descriptors/"
text_folder = "text_descriptors/"
vis_folder = "vis_descriptors/"
metadata_folder = "XML/"

train_entries_path = "res/CoeTraining.csv"

## Datasets

Features are built in the manner described in the paper of Wang & Liem or "Right Inflight? A Dataset for Exploring the Automatic
Prediction of Movies Suitable for a Watching Situation" (https://mmsys2016.itec.aau.at/papers/MMSYS/a45-riegler.pdf), if Wang & Liem do not provide any information.

This leads to following set-up:

Metadata: (language, year published, genre, country, runtime and age rating) - from XML<br>
Text: as is td-idf <br>
Audio: Averaged of all Frames (NaN to 0) - Mel-Frequency Cepstral Coefficients<br>
Visual: as is - Histogram of Oriented Gradients (HOG) gray, Color Moments, local binary patterns (LBP) and Gray Level Run Length Matrix

NOTE: Training data - invalid entry (2_states, also in test set), (Moulin_Rouge!.mp4, should be Moulin_Rouge! --> fixed)

In [51]:
df_base_train = pd.read_csv(train_entries_path)
df_base_train = df_base_train[df_base_train['file_name'] != '2_States'] # remove invalid entry
df_base_train.reset_index(inplace=True, drop=True)
df_base_train.sort_values(by='file_name', inplace=True)
df_targets_train = df_base_train['goodforairplanes'].astype(str)
df_base_train.head(5)

,movie_name,file_name,goodforairplanes
46,A Fish Called Wanda,A_Fish_Called_Wanda,1
93,A Goofy Movie,A_Goofy_Movie,0
85,A Million Ways to Die in the West,A_Million_Ways_to_Die_in_the_West,1
26,A Single Man,A_Single_Man,1
16,American Gangster,American_Gangster,1


## Feature extractors
As the dataset was built in a manner that would have been considered dirty already in 2002 a lot of feature extraction is done

In [6]:
def get_audio_features(file_name, use_train=True):
    """
        returns 1x14 dataframe, with averaged Mel-Frequency Cepstral Coefficients + file_name
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, audio_folder, file_name + ".csv")
    if not os.path.isfile(file_path):
        print(file_name, " does not exist!")
        return pd.DataFrame(columns=[str(x) for x in range(13)] + ['file_name'])
    df_audio = pd.read_csv(file_path, header=None).T # transpose (columns are rows)
    df_audio = df_audio.fillna(0) # nan values are treated as 0
    df_audio = pd.DataFrame(df_audio.mean(axis=0)).T # average accross columns
    df_audio['file_name'] = file_name
    return df_audio

def get_all_audio_features(df, use_train=True):
    """
        returns nx14 dataframe, containing audio features for all movies
    """
    dfs = []
    for file_name in df['file_name']:
        dfs.append(get_audio_features(file_name, use_train))
    
    return pd.concat(dfs)

def get_all_text_features(df, use_train=True):
    """
        returns nx3284 dataframe, containing tf-idf features for all movies
        the dataset creators messed up - contains several terms multiple times
        ordered alphabetically (?) - Live_Nude_Girls and Transformers__Age_of_Extinction where switched (detected perchance)
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, text_folder, "tdf_idf_dev.csv")
    df_txt = pd.read_csv(file_path)
    # the creators of the dataset missed how csv-files work - so we transpose and drop empty rows to get the correct format
    cols = df_txt.columns 
    df_txt = df_txt.T.dropna()
    df_txt.columns = cols
    df_txt.reset_index(inplace=True, drop=True)
    df_txt['file_name'] = sorted(df['file_name']) # we assume the info to be order alphabetically, as we do not have more info
    return df_txt

def get_vis_features(file_name, use_train=True):
    """
        returns 1x1653 dataframe, with unspecified visual features + file_name
        we assume that every single value in the csv is one feature
        this may be wrong, as there are two rows and no documentation (again)
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, vis_folder, file_name + ".csv")
    if not os.path.isfile(file_path):
        print(file_name, " does not exist!")
        return pd.DataFrame(columns=[str(x) for x in range(1652)] + ['file_name'])
    df_vis = pd.read_csv(file_path, header=None)
    df_vis = pd.DataFrame(pd.concat([df_vis.loc[0,:], df_vis.loc[1,:]])).reset_index(drop=True).T # treat each value as single feature (-> no aggregation)
    df_vis['file_name'] = file_name
    return df_vis

def get_all_vis_features(df, use_train=True):
    """
        returns nx1653 dataframe, containing visual features for all movies
    """
    dfs = []
    for file_name in df['file_name']:
        dfs.append(get_vis_features(file_name, use_train))
    
    return pd.concat(dfs)

def get_meta_features(file_name, use_train=True):
    """
        returns 1x7 dataframe, with metadata features + file_name
        One Hot Encoding is not applied here, this should happen later
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, metadata_folder, file_name + ".xml")
    if not os.path.isfile(file_path):
        print(file_name, " does not exist!")
        return pd.DataFrame(columns=['country', 'genre', 'language', 'rated', 'runtime', 'year', 'file_name'])
    etree = ET.parse(file_path)
    movie = etree.getroot().find('movie')
    mv = {}
    mv['language'] = [movie.get('language')]
    mv['year'] = [int(movie.get('year'))]
    mv['genre'] = [movie.get('genre')]
    mv['country'] = [movie.get('country')]
    mv['runtime'] = [int(movie.get('runtime')[:-4])]
    mv['rated'] = [movie.get('rated')]

    df_meta = pd.DataFrame.from_dict(mv)
    df_meta['file_name'] = file_name
    
    return df_meta

def get_all_meta_features(df, use_train=True):
    """
        returns nx7 dataframe, containing metadata features for all movies
    """
    dfs = []
    for file_name in df['file_name']:
        dfs.append(get_meta_features(file_name, use_train))
    
    return pd.concat(dfs)

In [45]:
df_audio_train = get_all_audio_features(df_base_train)
df_audio_train.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,file_name
0,66.828814,-5.674521,1.670346,1.143263,-0.635255,1.269376,0.633811,0.012407,0.164257,-1.494813,-0.292583,0.086875,-0.914582,A_Fish_Called_Wanda
0,43.908715,-6.380989,1.361790,-1.483354,-0.670040,-1.626537,0.466197,-1.888169,0.848654,-0.990286,0.673634,-0.972964,-0.141539,A_Goofy_Movie
0,3.390978,-6.725758,0.579762,-0.271885,-0.175640,-0.845690,-0.699064,-0.578434,0.537249,-1.387373,0.747223,-0.887580,-0.205273,A_Million_Ways_to_Die_in_the_West
0,57.743484,-3.722123,2.780418,0.756402,0.043743,-0.960622,-0.435575,-0.176729,1.665236,-2.068548,1.211791,-0.358194,0.738827,A_Single_Man
0,65.354709,-5.609515,-1.303409,-0.831993,-0.518848,-0.019373,-0.500203,-0.897985,0.148561,-0.666728,0.033135,0.383797,-0.209412,American_Gangster


In [46]:
df_txt_train = get_all_text_features(df_base_train)
df_txt_train.head(5)

,24000,baby,baseball,big,doc,escort,frozen,heroes,high,huck,...,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists,file_name
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A_Fish_Called_Wanda
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A_Goofy_Movie
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A_Million_Ways_to_Die_in_the_West
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A_Single_Man
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.051657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,American_Gangster


In [47]:
df_vis_train = get_all_vis_features(df_base_train)
df_vis_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,file_name
0,0.430310,0.38101,0.34082,0.31642,0.414650,0.385990,0.329380,0.31212,0.350670,0.342460,...,362.83,8.592300,9.142700,8.410100,8.792400,1483.3,417.21,892.59,435.28,A_Fish_Called_Wanda
0,0.002031,0.00000,0.00000,0.07302,0.027533,0.005346,0.006015,0.11824,0.026991,0.005171,...,20278.00,0.972010,1.365400,1.803200,1.463400,168740.0,20896.00,34434.00,20967.00,A_Goofy_Movie
0,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,119950.00,0.000001,0.002466,0.000004,0.002466,729320.0,119950.00,230400.00,119950.00,A_Million_Ways_to_Die_in_the_West
0,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,9550.70,5.943300,6.021500,4.383900,5.428800,32242.0,10313.00,22308.00,9850.00,A_Single_Man
0,0.231640,0.28629,0.30068,0.28118,0.244890,0.278490,0.290760,0.29380,0.161040,0.166850,...,109.62,17.237000,15.543000,13.671000,15.231000,53559.0,8637.40,18597.00,8679.00,American_Gangster


In [48]:
df_meta_train = get_all_meta_features(df_base_train)
df_meta_train.head(5)

,country,genre,language,rated,runtime,year,file_name
0,"USA, UK","Comedy, Crime","English, Italian, Russian",R,108,1988,A_Fish_Called_Wanda
0,USA,"Animation, Adventure, Comedy",English,G,78,1995,A_Goofy_Movie
0,USA,"Comedy, Western","English, Navajo, Mandarin",R,116,2014,A_Million_Ways_to_Die_in_the_West
0,USA,Drama,"English, Spanish",R,99,2009,A_Single_Man
0,"USA, UK","Biography, Crime, Drama",English,R,157,2007,American_Gangster


In [93]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB # put NOTE into paper - not sure if correct bayes
from sklearn.model_selection import KFold
from sklearn.base import clone

class BaseClassifier:
    
    def __init__(self, clf, clf_name, modality):
        self.clf = clf
        self.clf_name = clf_name
        self.modality = modality
        
    def fit(self, df_features, df_targets, verbose=True):
        self.clf = clone(self.clf) # reset any previously trained model
        np.random.seed(32143421)
        if verbose:
            print(f"Starting training for classifier {self.clf_name} and modality {self.modality}")
        self.clf.fit(df_features, df_targets)
        
    def predict(self, df_features, verbose=True):
        if verbose:
            print(f"Starting prediction for classifier {self.clf_name} and modality {self.modality}")
        return self.clf.predict(df_features)    
        
class ClassifierFactory:
    
    @staticmethod
    def get_metadata_classifiers():
        return [BaseClassifier(KNeighborsClassifier(), 'k-Nearest neighbors', 'Metadata'),
                BaseClassifier(NearestCentroid(), 'Nearest mean classifier', 'Metadata'),
                BaseClassifier(DecisionTreeClassifier(), 'Decision tree', 'Metadata'),
                BaseClassifier(LogisticRegression(), 'Logistic regression', 'Metadata'),
                BaseClassifier(SVC(gamma='auto'), 'SVM (Gaussian Kernel)', 'Metadata'),
                BaseClassifier(BaggingClassifier(),'Bagging', 'Metadata'),
                BaseClassifier(RandomForestClassifier(n_estimators=10),'Random Forest', 'Metadata'),
                BaseClassifier(AdaBoostClassifier(), 'AdaBoost', 'Metadata'),
                BaseClassifier(GradientBoostingClassifier(), 'Gradient Boosting Tree', 'Metadata')]
    
    @staticmethod
    def get_text_classifiers():
        return [BaseClassifier(GaussianNB(), 'Naive Bayes', 'Textual'),
                BaseClassifier(KNeighborsClassifier(), 'k-Nearest neighbors', 'Textual'),
                BaseClassifier(SVC(gamma='auto'), 'SVM (Gaussian Kernel)', 'Textual')]
    
    @staticmethod
    def get_visual_classifiers():
        return [BaseClassifier(KNeighborsClassifier(), 'k-Nearest neighbors', 'Visual'),
                BaseClassifier(DecisionTreeClassifier(), 'Decision tree', 'Visual'),
                BaseClassifier(LogisticRegression(), 'Logistic regression', 'Visual'),
                BaseClassifier(SVC(gamma='auto'), 'SVM (Gaussian Kernel)', 'Visual'),
                BaseClassifier(RandomForestClassifier(n_estimators=10), 'Random Forest', 'Visual'),
                BaseClassifier(AdaBoostClassifier(), 'AdaBoost', 'Visual'),
                BaseClassifier(GradientBoostingClassifier(), 'Gradient Boosting Tree', 'Visual')]
    
    @staticmethod
    def get_audio_classifiers():
        return [BaseClassifier(LogisticRegression(), 'Logistic regression', 'Audio'),
                BaseClassifier(GradientBoostingClassifier(), 'Gradient Boosting Tree', 'Audio')]                
                
class DataWrapper:
    
    def __init__(self, df_audio, df_vis, df_txt, df_meta, df_targets):
        self.df_audio = df_audio
        self.df_vis = df_vis
        self.df_txt = df_txt
        self.df_meta = df_meta
        self.df_targets = df_targets
        
    def generate_subspace(self):
        # TODO insert LVW functionality
        self.df_audio = self.df_audio.drop('file_name', axis=1)
        self.df_vis = self.df_vis.drop('file_name', axis=1)
        self.df_txt = self.df_txt.drop('file_name', axis=1)
        self.df_meta = self.df_meta.drop('file_name', axis=1)
        
        
class Evaluator:
    # TODO add clf stacking method
    
    def __init__(self, data_wrapper, use_audio=True, use_visual=True, use_text=True, use_meta=True):
        self.data_wrapper = data_wrapper
        self.use_audio = use_audio
        self.use_visual = use_visual
        self.use_text = use_text
        self.use_meta = use_meta            
            
    def cv_modality(self, df_features, df_targets, clfs, cv=10, verbose=True):
        kf = KFold(n_splits=cv, random_state=2109432)
        df_cvs = []
        i=1
        for train_index, test_index in kf.split(df_features):
            if verbose:
                print(f"Performing CV fold {i}..")
            i += 1
            X_train, X_test = df_features.iloc[train_index,:], df_features.iloc[test_index,:]
            y_train, y_test = df_targets[train_index], df_targets[test_index]
            df_res = pd.DataFrame(y_test)
            df_res.columns = ['TARGET']
            for clf in clfs:
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                df_res[clf.clf_name+"_"+clf.modality] = y_pred
            df_cvs.append(df_res)
        return df_cvs
            
    def cv(self, cv=10, verbose=True):
        # TODO: merge results per fold
        df_cvs = []
        df_targets = self.data_wrapper.df_targets
        if (self.use_audio):
            df_cvs.append(self.cv_modality(
                self.data_wrapper.df_audio,
                df_targets,
                ClassifierFactory.get_audio_classifiers(),
                cv,
                verbose))
        if (self.use_visual):
            df_cvs.append(self.cv_modality(
                self.data_wrapper.df_vis,
                df_targets,
                ClassifierFactory.get_visual_classifiers(),
                cv,
                verbose))
        if (self.use_text):
            df_cvs.append(self.cv_modality(
                self.data_wrapper.df_txt,
                df_targets,
                ClassifierFactory.get_text_classifiers(),
                cv,
                verbose))
        if (self.use_meta):
            df_cvs.append(self.cv_modality(
                self.data_wrapper.df_meta,
                df_targets,
                ClassifierFactory.get_metadata_classifiers(),
                cv,
                verbose))
        return df_cvs
        
        

In [94]:
dw = DataWrapper(df_audio_train, df_vis_train, df_txt_train, df_meta_train, df_targets_train)
dw.generate_subspace()
ev = Evaluator(dw, use_meta=False) # TODO preprocessing for meta needs to be done
df_cvs = ev.cv()

Performing CV fold 1..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio
Starting prediction for classifier Gradient Boosting Tree and modality Audio
Performing CV fold 2..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio
Starting prediction for classifier Gradient Boosting Tree and modality Audio
Performing CV fold 3..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio


E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Starting prediction for classifier Gradient Boosting Tree and modality Audio
Performing CV fold 4..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio
Starting prediction for classifier Gradient Boosting Tree and modality Audio
Performing CV fold 5..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio
Starting prediction for classifier Gradient Boosting Tree and modality Audio
Performing CV fold 6..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio
Starting prediction for classifier Gr

E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Starting prediction for classifier Gradient Boosting Tree and modality Audio
Performing CV fold 8..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio
Starting prediction for classifier Gradient Boosting Tree and modality Audio
Performing CV fold 9..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio
Starting prediction for classifier Gradient Boosting Tree and modality Audio
Performing CV fold 10..
Starting training for classifier Logistic regression and modality Audio
Starting prediction for classifier Logistic regression and modality Audio
Starting training for classifier Gradient Boosting Tree and modality Audio
Starting prediction for classifier G

...

Starting prediction for classifier AdaBoost and modality Visual
Starting training for classifier Gradient Boosting Tree and modality Visual
Starting prediction for classifier Gradient Boosting Tree and modality Visual
Performing CV fold 9..
Starting training for classifier k-Nearest neighbors and modality Visual
Starting prediction for classifier k-Nearest neighbors and modality Visual
Starting training for classifier Decision tree and modality Visual
Starting prediction for classifier Decision tree and modality Visual
Starting training for classifier Logistic regression and modality Visual
Starting prediction for classifier Logistic regression and modality Visual
Starting training for classifier SVM (Gaussian Kernel) and modality Visual
Starting prediction for classifier SVM (Gaussian Kernel) and modality Visual
Starting training for classifier Random Forest and modality Visual
Starting prediction for classifier Random Forest and modality Visual
Starting training for classifier AdaBoo

E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Python\Miniconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Starting prediction for classifier AdaBoost and modality Visual
Starting training for classifier Gradient Boosting Tree and modality Visual
Starting prediction for classifier Gradient Boosting Tree and modality Visual
Performing CV fold 10..
Starting training for classifier k-Nearest neighbors and modality Visual
Starting prediction for classifier k-Nearest neighbors and modality Visual
Starting training for classifier Decision tree and modality Visual
Starting prediction for classifier Decision tree and modality Visual
Starting training for classifier Logistic regression and modality Visual
Starting prediction for classifier Logistic regression and modality Visual
Starting training for classifier SVM (Gaussian Kernel) and modality Visual
Starting prediction for classifier SVM (Gaussian Kernel) and modality Visual
Starting training for classifier Random Forest and modality Visual
Starting prediction for classifier Random Forest and modality Visual

E:\Python\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Python\Miniconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)



Starting training for classifier AdaBoost and modality Visual
Starting prediction for classifier AdaBoost and modality Visual
Starting training for classifier Gradient Boosting Tree and modality Visual
Starting prediction for classifier Gradient Boosting Tree and modality Visual
Performing CV fold 1..
Starting training for classifier Naive Bayes and modality Textual
Starting prediction for classifier Naive Bayes and modality Textual
Starting training for classifier k-Nearest neighbors and modality Textual
Starting prediction for classifier k-Nearest neighbors and modality Textual
Starting training for classifier SVM (Gaussian Kernel) and modality Textual
Starting prediction for classifier SVM (Gaussian Kernel) and modality Textual
Performing CV fold 2..
Starting training for classifier Naive Bayes and modality Textual
Starting prediction for classifier Naive Bayes and modality Textual
Starting training for classifier k-Nearest neighbors and modality Textual
Starting prediction for cla

In [96]:
df_cvs[0][0]

,TARGET,Logistic regression_Audio,Gradient Boosting Tree_Audio
0,1,0,0
1,0,1,0
2,0,1,1
3,0,0,1
4,1,1,1
5,1,0,0
6,1,1,1
7,1,1,1
8,1,1,1
9,1,1,1
